In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = 'EleutherAI/pythia-160m'
model_name = 'EleutherAI/pythia-70m'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model_og = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AAAAAA 16
AAAAAA 16
AAAAAA 16
AAAAAA 16
AAAAAA 16
AAAAAA 16


In [2]:
!python auto_gen_components.py

Traceback (most recent call last):
  File "/home/lev/code/research/dictator_what/maraboupy_usage/auto_gen_components.py", line 254, in <module>
    gen_attn_head_unrolled('tmp/attn_head.py', 'tmp.qkv_copier',
TypeError: gen_attn_head_unrolled() missing 1 required positional argument: 'rotary_embed_path'


In [ ]:
def get_input_after_tokenizer(inp: str):
		tokenizer = AutoTokenizer.from_pretrained(model_name)
		inp_out = tokenizer(inp, return_tensors="pt")
		print(inp_out)
		inputs = inp_out['input_ids'].squeeze(0)
		# print(inputs, inputs.shape)
		# TODO: IDK ABOUT WHATS GOING ON W/ TOKEN SIZE VS Vocab Size
		# Vocab size is 50204 and inp size is 50304
		one_hot = torch.zeros((inputs.shape[0], 50304), dtype=torch.int)
		for i in range(inputs.shape[0]):
			one_hot[i, inputs[i]] = 1
		# one_hot[inputs['input_ids'][0, 0]] = 1
		return one_hot#, inp_out['attention_mask']
get_input_after_tokenizer('hello world')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [ ]:
inps_one_hot = get_input_after_tokenizer("hello world")
seq_len = inps_one_hot.shape[0]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


In [ ]:
model_og.gpt_neox.layers[0]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_dropout): Dropout(p=0.0, inplace=False)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
    (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
    (act): GELUActivation()
  )
)

In [ ]:
from simplified import ModelSel

model_sel = ModelSel(model_og)
model_sel

ModelSel(
  (embed_linear): Linear(in_features=50304, out_features=512, bias=False)
  (layer_norm): SimplfiedLayerNorm(
    (eps): Linear(in_features=512, out_features=512, bias=True)
    (ones_linear): Linear(in_features=512, out_features=512, bias=False)
    (ones_linear_neg): Linear(in_features=512, out_features=512, bias=False)
  )
  (attn): FixedAttentionMask(
    (attn): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
      (dense): Linear(in_features=512, out_features=512, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
    )
    (slice_query): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_value): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_key): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_rotary): Slicer(
      (slicer): Li

In [ ]:
inps_one_hot_formatted = inps_one_hot.float()
# embed_linear.forward(inps_one_hot).shape
model_sel(inps_one_hot_formatted).shape

torch.Size([2, 1536])
QKV SIZE torch.Size([2, 1536]) torch.Size([2])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size(

torch.Size([2, 512])

## Convert to Tensorflow

In [ ]:
model_og?

Signature:      model_og(*args, **kwargs)
Type:           GPTNeoXForCausalLM
String form:   
GPTNeoXForCausalLM(
           (gpt_neox): GPTNeoXModel(
           (embed_in): Embedding(50304, 512)
           (emb_dr <...> entwise_affine=True)
           )
           (embed_out): Linear(in_features=512, out_features=50304, bias=False)
           )
File:           ~/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py
Docstring:     
GPTNeoX Model with a `language modeling` head on top for CLM fine-tuning.
This model is a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) sub-class. Use
it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage and
behavior.

Parameters:
    config ([`~GPTNeoXConfig`]): Model configuration class with all the parameters of the model.
        Initializing with a config file does not load the weights as

In [ ]:
import nobuco
from nobuco import ChannelOrder, ChannelOrderingStrategy
from nobuco.layers.weight import WeightLayer

pytorch_module = model_sel.eval()

dummy_image = inps_one_hot_formatted
keras_model = nobuco.pytorch_to_keras(
    pytorch_module,
    args=[dummy_image], kwargs=None,
    inputs_channel_order=ChannelOrder.TENSORFLOW,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)

2024-03-16 20:35:14.551822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 20:35:14.551859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 20:35:14.553373: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 20:35:14.563187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 20:35:15.871714: W tensorflow/compiler/tf2

torch.Size([2, 1536])
QKV SIZE torch.Size([2, 1536]) torch.Size([2])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size(

/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/converters/validation.py:66: UserWarning: Validation exception on node 'GPTNeoXRotaryEmbedding': Unimplemented op: GPTNeoXRotaryEmbedding()
  warnings.warn(f"Validation exception on node '{op_type.__name__}': {e}")
Traceback (most recent call last):
  File "/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/converters/validation.py", line 47, in validate
    diffs = validate_diff_default(keras_op, pytorch_op, input_args, input_kwargs, output_tensors)
  File "/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/converters/validation.py", line 75, in validate_diff_default
    outputs_tf = keras_op(*args_tf, **kwargs_tf)
  File "/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/layers/stub.py", line 7, in __call__
    raise Exception(f'Unimplemented op: {self.ori

/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/converters/validation.py:66: UserWarning: Validation exception on node 'FixedAttentionMask': Unimplemented op: GPTNeoXRotaryEmbedding()
  warnings.warn(f"Validation exception on node '{op_type.__name__}': {e}")
Traceback (most recent call last):
  File "/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/converters/validation.py", line 47, in validate
    diffs = validate_diff_default(keras_op, pytorch_op, input_args, input_kwargs, output_tensors)
  File "/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/converters/validation.py", line 75, in validate_diff_default
    outputs_tf = keras_op(*args_tf, **kwargs_tf)
  File "/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/layers/container.py", line 69, in __call__
    outputs = op(*args, **kwargs)
  File "/home

Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

 I  File "/home/lev/code/research/dictator_what/maraboupy_usage/.venv/lib/python3.10/site-packages/nobuco/trace/trace.py", line 429 
 D  File "/home/lev/code/research/dictator_what/maraboupy_usage/simplified.py", line 35  
ModelSel[simplified](float32_0<2,50304>) -> float32_367<2,512>
 │  Linear[torch.nn.modules.linear](float32_0<2,50304>) -> float32_2<2,512>
 │   └· linear[torch.nn.functional](float32_0<2,50304>, float32_1<512,50304>, None) -> float32_2<2,512>
 │  SimplfiedLayerNorm[simplified](float32_2<2,512>) -> float32_14<2,512>
 │   │  Linear[torch.nn.modules.linear](float32_2<2,512>) -> float32_4<2,512>
 │   │   └· linear[tor

Exception: Unimplemented nodes